In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [2]:
url = "https://data.ca.gov/dataset/590188d5-8545-4c93-a9a0-e230f0db7290/resource/926fd08f-cc91-4828-af38-bd45de97f8c3/download/statewide_cases.csv"

In [3]:
df = pd.read_csv(url)

In [4]:
df.head()

,county,totalcountconfirmed,totalcountdeaths,newcountconfirmed,newcountdeaths,date
0,Santa Clara,151.0,6.0,151,6,2020-03-18
1,Santa Clara,183.0,8.0,32,2,2020-03-19
2,Santa Clara,246.0,8.0,63,0,2020-03-20
3,Santa Clara,269.0,10.0,23,2,2020-03-21
4,Santa Clara,284.0,13.0,15,3,2020-03-22


In [5]:
rds_connection_string = "postgres:postgres@localhost:5432/covid-vaccine-plz"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [6]:
engine.table_names()

[]

In [7]:
df.to_sql(name='statewide_table',con=engine,if_exists='append',index=False)

In [8]:
pd.read_sql_query('select* from statewide_table',con=engine).head()

,county,totalcountconfirmed,totalcountdeaths,newcountconfirmed,newcountdeaths,date
0,Santa Clara,151.0,6.0,151,6,2020-03-18
1,Santa Clara,183.0,8.0,32,2,2020-03-19
2,Santa Clara,246.0,8.0,63,0,2020-03-20
3,Santa Clara,269.0,10.0,23,2,2020-03-21
4,Santa Clara,284.0,13.0,15,3,2020-03-22


In [10]:
df.to_csv("./CSV/statewide.csv", index=False)